In [11]:
#This program fits the O-PPAC data onto a FCNN model
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, metrics, Model
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split
import tensorflow_docs.plots
import tensorflow_docs.modeling
import numpy as np
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import h5py
import datetime

In [ ]:
dataset = np.load("/home/roberts/O-PPAC/recomplied_dataset_fixed.npy")
all_data = dataset[ : , :132]
all_labels = dataset[: , -2:]

print(all_data.shape)
print(all_labels.shape)

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(all_data, all_labels, test_size=0.2,shuffle=True)
#splits training and test data for the FCNN; 20% set aside for testing

In [ ]:
print(train_data.ndim)
print(train_data.shape)
print(train_data[0])

In [ ]:
#early stopping mechanism; can increase amount of time before stopping by increasing patience
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
#Fully connected neural network
#the "layers" are the dense hidden layers which are fully connected
#first number changes the nodes, 2nd is the activation function, (first line has number of inputs as well)

FCNN_model = tf.keras.Sequential()
FCNN_model.add(layers.Dense(256, activation='relu', input_shape = [132]))
FCNN_model.add(layers.Dense(256, activation = 'relu'))
#FCNN_model.add(layers.Dense(128, activation = 'relu'))
#FCNN_model.add(layers.Dense(128, activation = 'relu'))
#FCNN_model.add(layers.Dropout(0.2))
FCNN_model.add(layers.Dense(2, activation = 'linear'))

In [ ]:
#compiles the FCNN and includes the learning rate as well
FCNN_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss = 'mse', 
              metrics=['mae', 'mse'])

In [ ]:
FCNN_history = FCNN_model.fit(train_data, train_labels,
                    epochs=10,
                    batch_size = 32,
                    #callbacks = [callback],
                    validation_split = 0.3 
                    )

In [ ]:
#loss plot of training and test data
plt.plot(FCNN_history.history['loss'], label='loss')
plt.plot(FCNN_history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.yscale('log')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title("FCNN O-PPAC Loss Curve, Dropout 0, 2 Dense 256 Node, Adam 0.001, MSE, linear activation")
plt.show()

In [ ]:
# serialize model to JSON
FCNNmodel = FCNN_model.to_json()
with open("FCNNmodel", "w") as json_file:
    json_file.write(FCNNmodel)
# serialize weights to HDF5
FCNN_model.save_weights("FCNNmodel_weights.h5")

In [ ]:
#to reload model from before:

    # load json and create model
#model_file = open('FCNN', 'r')
#loaded_FCNN_model = model_file.read()
#model_file.close()
#FCNN_model = model_from_json(loaded_FCNN_model)

    # load weights into new model
#FCNN_model.load_weights("FCNNmodel_weights.h5")

In [ ]:
#separation of 'result' values (stats from test data)
result = FCNN_model.evaluate(test_data, test_labels, verbose=2)

In [ ]:
#separation of 'prediction' values (training data)
predictions = FCNN_model.predict(test_data)

In [ ]:
#separates x and y predicted values so there can be separate graphs for each
x_pred = predictions[:,0]
y_pred = predictions[:,1]

#separates x and y labels for comparisons to the predicted locations
x_test = np.array(test_labels[:,0])
y_test = np.array(test_labels[:,1])

In [ ]:
#graph showing the accuracy of model by comparing predicted and true x values
#should be as close to the diagonal as possible
a = plt.axes(aspect='equal')
plt.scatter(x_test, x_pred, s=6, marker = "x")
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [-50, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims, linewidth = .5)
plt.title("FCNN O-PPAC X Predictions vs True X Values")
plt.show()

In [ ]:
#graph showing the accuracy of model by comparing predicted and true y values
#should be as close to the diagonal as possible
a = plt.axes(aspect='equal')
plt.scatter(y_test, y_pred, s=6, marker = "x")
plt.xlabel('True Values')
plt.ylabel('Predictions')
lims = [-50, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims, linewidth = .5)
plt.title("FCNN O-PPAC Y Predictions vs True Y Values")
plt.show()

In [ ]:
#to find the locations of largest error within my predicted x values
x_error = x_pred - x_test

#this takes all the points with error greater than 1 mm
xerror_list = [err for (idx, err) in enumerate(x_error) if err >= 1]
print(xerror_list)

In [ ]:
#to find the locations of largest error within my predicted y values (same as the x code above but specific to y)
y_error = y_pred - y_test 

yerror_list = [ err for (idx, err) in enumerate(y_error) if err >= 1]
print(yerror_list)

In [ ]:
# setting up for residue plot of x-error from the model
error_list_x = [i for i in x_error if abs(i) <= .5]
print(len(error_list_x))

In [ ]:
# setting up for residue plot of y-error from the model
error_list_y = [i for i in y_error if abs(i) <= .5]
print(len(error_list_y))

In [12]:
def gaussianpdf(x, k, sigma, mu):
    return k*np.exp(-0.5*((x-mu)/sigma)**2)

def full_half(data):
    bin_heights, bin_borders, _ = plt.hist(data, bins='auto', label='Residual')
    bin_centers = bin_borders[:-1] + np.diff(bin_borders) / 2
    popt, _ = curve_fit(gaussianpdf, bin_centers, bin_heights, p0=[60., 10., 30.])
    fwhm = popt[1]*2.355
    
    x_interval_for_fit = np.linspace(bin_borders[0], bin_borders[-1], 10000) #the x interval the fit is put over
    plt.plot(x_interval_for_fit, gaussianpdf(x_interval_for_fit, *popt), label='Gaussian Fit')
    gauss = gaussianpdf(x_interval_for_fit, *popt) #y data for the gaussian fit if needed to be plotted separately
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    return np.abs(fwhm)

In [9]:
x_fwhm = full_half(error_list_x)
plt.title('FCNN X Residual')
plt.show()
print(x_fwhm)

NameError: name 'full_half' is not defined

In [ ]:
y_fwhm = full_half(error_list_y)
plt.title('FCNN Y Residual')
plt.show()
print(y_fwhm)